# Predict Rents for Munich

Task: Download a collection of immoscout24 data for rents in Germany from kaggle at https://www.kaggle.com/corrieaar/apartment-rental-offers-in-germany and reduce it to the city of Munich.
Examine which characteristics have a measurable influence on the rent.
Create a prediction model that predicts the required rent as accurately as possible for given data.
Also create visualizations of interesting relationships.
Compare different methods for the inclusion of non-metric features and different algorithms.

## Preparations

### Global imports and setings.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os
from collections import Counter

# To plot pretty figures
%matplotlib inline
import matplotlib.pyplot as plt

# data processing
import pandas as pd
!pip install pandas_profiling
import pandas_profiling
!pip install sklearn_pandas
from sklearn_pandas import DataFrameMapper

# additional regressors
!pip install xgboost
from xgboost import XGBRegressor
!pip install lightgbm
from lightgbm import LGBMRegressor

# prediction application
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import TwoByTwoLayout

# to make this notebook's output stable across runs
np.random.seed(42)

In [ ]:
# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "regession"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

### Dataset

#### Load

In [ ]:
# extract data from archive if not done already
if not os.path.isfile('immo_data.csv'):
    import zipfile
    zipfile.ZipFile('apartment-rental-offers-in-germany.zip', 'r').extractall()

In [ ]:
# load/read data
df = pd.read_csv("immo_data.csv")
df_origin = df.copy()

In [ ]:
print("appartements: %d" % len(df.values))
print("attributes: %d" % len(df.columns))

In [ ]:
report_enabled = True
report = lambda df : report_enabled and df.reset_index(drop=True).profile_report(title='Team Munich')

In [ ]:
# display all avalible data columns
df.columns

In [ ]:
# munich only
df = df[df.regio2=='München']

In [ ]:
# show head
df.head(5)

In [ ]:
df.describe()

#### Analyze

Let first detect the difference between `totalRent` and `baseRent`.

In [ ]:
plt.scatter(x='totalRent', y='baseRent', data=df)
plt.title('$totalRent$ vs $baseRent$')
plt.xlabel('totalRent')
plt.ylabel('baseRent')
plt.show()

As we can see in the plot, there is no relevant difference in behaviour between `totalRent` and `baseRent`. Therefore the behavior will be almost identically in predictions.

We choose `totalRent` as the basis feature.

In [ ]:
# if totalRent is not set use baseRent
df['totalRent'][df['totalRent'] <= 0] = np.NaN
df['totalRent'][np.isnan(df['totalRent'])] = (
    df['baseRent'][np.isnan(df['totalRent'])]
    + df['heatingCosts'][np.isnan(df['totalRent'])]
    + df['serviceCharge'][np.isnan(df['totalRent'])]
)

In [ ]:
# analyze feature importance
corr_matrix = df.corr()
#corr_matrix['totalRent'].sort_values(ascending=False)

In [ ]:
import seaborn as sns

plt.figure(figsize=(8, 12))
heatmap = sns.heatmap(corr_matrix[['totalRent']].sort_values(by='totalRent', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Features Correlating with totalRent', fontdict={'fontsize':18}, pad=16);

The important features for the `totalRent` prediction are:

- baseRent
- livingSpaceRange
- baseRentRange
- noRooms
- noRoomsRange
- serviceCharge
- heatingCosts

**Note**: It is important to know that the `corr_matrix`, or maybe it's more likely to say the correlation function of the pandas data frame, excludes values with `NaN` and `null`.

> Compute pairwise correlation of columns, excluding NA/null values.

Source: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html

But as we already know, the `baseRent` is almost identical to `totalRent`. Therefore we also drop this feature. If we do not drop `baseRent` the prediction would based on itself which would falsify the result. Because `baseRentRange` is highly correlated with `baseRent`, we will drop `baseRentRange` as well.

In other words if you keep y in X the prediction will only base on the feature y.

Instead of the `-Range` features, we want to take the base features of them because we don't know how the `-Range` features were calculated.
For this we need to confirm that `-Range` and base features are correlating with each others.

Hence, this are the remaining features:

- livingSpace
- livingSpaceRange
- noRooms
- noRoomsRange
- serviceCharge
- heatingCosts

In [ ]:
df = df.drop(['baseRent', 'baseRentRange'], axis=1)

To measure the association between the features and `totalRent` without using one-hot encode category features we will use Cramers V statistic calculation. For example we do not need to use one-hot encoding for `geo_plz`.

More inforamtion about [Cramers V](https://en.wikipedia.org/wiki/Cram%C3%A9r%27s_V) could be found in it's Wikipedia article.

In [ ]:
from scipy import stats

def cramers_v(x, y):
    """
    Cramers V statistic to calculate the measure of association
    without one-hot encoding the category variable.
    """
    confusion_matrix = pd.crosstab(x, y)
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
    rcorr = r-((r-1)**2)/(n-1)
    kcorr = k-((k-1)**2)/(n-1)
    return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))

In [ ]:
importance = {}
for feature in df.columns:
    importance[feature] = cramers_v(df[feature], df['totalRent'])

#pd.DataFrame(importance, index=[0]).T.sort_values(by=0, ascending=False) #.head(15)

In [ ]:
plt.figure(figsize=(8, 12))
heatmap = sns.heatmap(pd.DataFrame(importance, index=[0]).T.sort_values(by=0, ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Cramers V: Features Importance', fontdict={'fontsize':18}, pad=16);

Even Cramers V statistic provides that `geo_plz` does not associate with `totalRent`. Cramers V statistic also does not show any other suprises regarding the correlate with `totalRent`.

To examine the feature importance in more detail, we decided to also look at the Random Forest additionally to the correlation.

In [ ]:
# copy data set
X_forest = df.copy()

In [ ]:
# remove NaN values for feature important test
X_forest = X_forest.replace('N/A',np.NaN)
X_forest = X_forest.replace(np.NaN, 0)
X_forest.head(5)

Hence `RandomForestRegressor` can not operate well on strings, we will use a `LabelEncoder` first.

In [ ]:
# get inoperable columns
cols = X_forest.select_dtypes(include='object').columns
cols

In [ ]:
from sklearn.preprocessing import LabelEncoder

les = {}

original_tmp = X_forest
mask = X_forest.isnull()  # NaN

for col in cols:
    X_forest[col] = X_forest[col].astype(str)
    les[col] = LabelEncoder()
    X_forest[col] = les[col].fit_transform(X_forest[col])

X_forest = X_forest.where(~mask, original_tmp)
X_forest[cols]

In [ ]:
# to inverse the Label-Encoding
def le_inverse(col, df):
    if (isinstance(col, list)):
        return le_inverse_arr(col, df)

    if not col in les:
        return df[col]

    return les[col].inverse_transform(df[col])

def le_inverse_arr(cols, df):
    res = []
    for col in cols:
        res.append(le_inverse(col, df))
    return res

In [ ]:
le_inverse('heatingType', X_forest)

Since we just used Label-Encoding, we can take a short look at feature selection with `SelectKBest`.

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression
X = X_forest.drop('totalRent', axis= 1)
y = X_forest['totalRent']

columns = []

for k in range(1, len(X.columns)+1):
    fiiiit = SelectKBest(f_regression, k=k).fit(X, y)
    X_new = fiiiit.transform(X)
    fiiiit.get_support()

    cur_columns = X.columns[fiiiit.get_support()]

    print(f"{k:2}. {list(set(cur_columns)-set(columns))[0]}")
    columns = cur_columns

No surprises so far.

Now we are prepared for the `RandomForestRegressor` operation.

In [ ]:
# split y and X
y_forest = X_forest['totalRent']
X_forest = X_forest.drop('totalRent', axis=1)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=42)
forest = forest.fit(X_forest, y_forest)

importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
indices = np.argsort(importances)[::-1]
importances

In [ ]:
# print the feature ranking
print("Feature ranking")
print("===============")
for f in range(X_forest.shape[1]):
    print(f"{(f+1):2}.  {X_forest.columns[indices[f]]:<30} {importances[indices[f]]}")

In [ ]:
# Plot imporatance (Top 10)
feat_importances = pd.Series(importances, index=X_forest.columns)
feat_importances.nlargest(10).plot(kind='barh', title='Feature Importance')

Hence that the feature `geo_plz` and `picturecount` are important in the last plots, we decide to keep them.

In [ ]:
# check skewness of features
skewness_for_features = [df[feature].skew(skipna=True)
                         for feature in df.select_dtypes(exclude='object').columns]

fig = plt.figure(figsize=(12, 6))
plt.bar(df.select_dtypes(exclude='object').columns, skewness_for_features, figure=fig)
plt.xticks(rotation=90)
plt.title('Skewness')
plt.show()

In [ ]:
df_skeweness = df.copy()
df_skeweness['noParkSpaces']  = np.log(df_skeweness['noParkSpaces'])
df_skeweness['livingSpace']   = np.log(df_skeweness['livingSpace'])
df_skeweness['lastRefurbish'] = np.log(df_skeweness['lastRefurbish'])
df_skeweness['geo_plz']       = np.log(df_skeweness['geo_plz'])

In [ ]:
corr_matrix = df_skeweness[['noParkSpaces', 'livingSpace', 'geo_plz', 'lastRefurbish', 'totalRent']].corr()
corr_matrix['totalRent'].sort_values(ascending=False)

As we can see, there is a high correlation between `livingSpace` and `totalRent`.
So we take `livingSpace` as a feature.

Lets have a look if `noParkSpaces` is also a good feature.

In [ ]:
df['noParkSpaces']

In [ ]:
# count NaN values
len(df['noParkSpaces'][np.isnan(df['noParkSpaces'])])

The feature`noParkSpaces` is ~44.2% not set.

In [ ]:
# show unique types
set(df['noParkSpaces'][~np.isnan(df['noParkSpaces'])])

In [ ]:
len(df['noParkSpaces'][df['noParkSpaces'] > 4])

There are seven appartements which have an unrealistic number of parking spaces.
We decide to remove them from our dataset.

Furthermore, we decide to set all appartments which doesn't set the `noParkSpaces` to zero, because by default an appartement in Munich doesn't provide an own parking space.

In [ ]:
df = df.drop(df['noParkSpaces'][df['noParkSpaces'] > 4].index)
df['noParkSpaces'] = df['noParkSpaces'].fillna(0)

In [ ]:
df['noParkSpaces'].describe()

Now to check if we should add `noParkSpaces` lets check the correlation again.

In [ ]:
# remove skeweness
df['noParkSpaces'] = np.log(df['noParkSpaces'])

# show correlation
corr_matrix = df[['noParkSpaces', 'totalRent']].corr()
corr_matrix['totalRent'].sort_values(ascending=False)

We will add `noParkSpaces` as a feature.

In [ ]:
# revert skeweness
df['noParkSpaces'] = np.exp(df['noParkSpaces'])

In [ ]:
# remaining features
features = [
    'totalRent',   # use this feature as y later

    'livingSpace',
    'livingSpaceRange',
    'noRooms',
    'noRoomsRange',
    'serviceCharge',
    'noParkSpaces',

    'geo_plz',
    'picturecount',

    # for feature engeneering testing
    #'heatingCosts',
    #'condition',
    #'heatingType',
]

Now we will drop all features which we do operate with anymore.

In [ ]:
# drop all other features
df = df.drop(df.columns.difference(features), axis=1)

In [ ]:
df_plot = df.copy()
df_plot['livingSpace'] = np.log(df_plot['livingSpace'])  # remove skeweness
report(df_plot)

Out of the report we can see that `-Range` features are correlated with the base features

Therefore, we are using only the base features.

In [ ]:
df = df.drop(['livingSpaceRange', 'noRoomsRange'], axis=1)
features.remove('livingSpaceRange')
features.remove('noRoomsRange')
print(features)
df

# Data Cleaning

To better understand what the features mean, we could read their descripion from the source.

features|description
--|--
totalRent         | total rent (usually a sum of base rent, service charge and heating cost)
livingSpace       | living space in sqm
noRooms           | number of rooms
serviceCharge     | aucilliary costs such as electricty or internet in €
noParkSpaces      | number of parking spaces
gep_plz           | ZIP code
picturecount      | how many pictures were uploaded to the listing

In [ ]:
df.describe()

In [ ]:
report(df)

If we look for example at the `serviceCharge` feature in the report, we see there are some appartements in the dataset which does not provide us a service charge value. Naturally, this could also be the case for other features.

Values which are processable but not applicable are the most dangerous ones. We need to choose the best values or even remove these appartements from our prediction dataset.

Special attention must be paid to the values `NaN`, `null` and `0`!

In the most cases there are three default approches how to replace `NaN` and invalid values.

Here an example based on `serviceCharge`:

- We could assume that no additional ancilliary costs such as electricty or internet are incurred in these cases. => `0`
- We could choose the average/mean.
- We could choose the median.



For `serviceCharge` we choose to set all Nan and negative values to the mean of all serviceCharges

In [ ]:
## serviceCharge

# set NaN and negative values to zero
df['serviceCharge'][df['serviceCharge'] < 0] = 0
df['serviceCharge'][np.isnan(df['serviceCharge'])] = np.mean(df['serviceCharge'])

In [ ]:
## noRooms

# set invalid values to NaN
df['noRooms'][df['noRooms'] < 1] = np.NaN
df['noRooms'][df['noRooms'] > 1000] = np.NaN

# calculate mean of valid values
mean = int(np.mean(df['noRooms'][np.isnan(df['noRooms']) == False]))

# set NaN to median
df['noRooms'][np.isnan(df['noRooms'])] = mean

In [ ]:
## heatingCosts

# set NaN and negative values to zero
#df['heatingCosts'][np.isnan(df['heatingCosts'])] = 0
#df['heatingCosts'][df['heatingCosts'] < 0] = 0

# calculate mean of none zero values
#mean = np.mean(df['heatingCosts'][df['heatingCosts'] != 0])

# set zero values to mean
#df['heatingCosts'][df['heatingCosts'] == 0] = mean

In [ ]:
## livingSpace

# remove outliner
df = df.drop(df['livingSpace'][df['livingSpace'] > 1000].index)

# skeweness
#from sklearn.preprocessing import MinMaxScaler
#livingSpace_scaler = MinMaxScaler()
#df['livingSpace'] = livingSpace_scaler.fit_transform(df['livingSpace'].values.reshape(-1,1))

Note: We have nothing to do for `noParkSpaces` regarding invalid large parking spaces anymore.

In [ ]:
## noParkSpaces

# set NaN to invalid appatements
df['noParkSpaces'][df['noParkSpaces'] < 0] = np.NaN

# calculate mean
mean = int(np.mean(df['noParkSpaces'][np.isnan(df['noParkSpaces']) == False]))

# set invalid values to mean
df['noParkSpaces'][np.isnan(df['noParkSpaces'])] = mean

# skeweness
#from sklearn.preprocessing import MinMaxScaler
#noParkSpaces_scaler = MinMaxScaler()
#df['noParkSpaces'] = noParkSpaces_scaler.fit_transform(df['noParkSpaces'].values.reshape(-1,1))

In [ ]:
## geo_plz

# drop all invalid post codes in Munich.
df = df.drop(df['geo_plz'][df['geo_plz'] < 79999].index)
df = df.drop(df['geo_plz'][df['geo_plz'] > 82000].index)

In [ ]:
## totalRent

# remove NaN
df = df[df['totalRent'].notna()]

# remove outliner
df = df.drop(df['totalRent'][df['totalRent'] > 5000].index)

We remove invalid appartments and remove duplicated rows.

In [ ]:
## picturecount
df['picturecount'] = df['picturecount'].fillna(int(df['picturecount'].mean()))

In [ ]:
df = df.reset_index(drop=True)       # remove index
df = df.drop_duplicates(keep=False)  # remove dupplicated rows

# Feature Engineering

We try to create a new feature that is more correlated to `totalRent` by adding `serviceCharge` and `heatingCosts`.

In [ ]:
#df['serviceChargeAndHeatingCosts'] = df['serviceCharge'] + df['heatingCosts']

In [ ]:
#corr_matrix = df[['serviceChargeAndHeatingCosts', 'serviceCharge', 'heatingCosts', 'totalRent']].corr()
#corr_matrix['totalRent'].sort_values(ascending=False)

```
totalRent                       1.000000
serviceChargeAndHeatingCosts    0.529311
serviceCharge                   0.501365
heatingCosts                    0.293588
Name: totalRent, dtype: float64
```

In [ ]:
#df = df.drop(['serviceCharge', 'heatingCosts'], axis=1)

#### <span style="color: #FF8888">**Unfortunately, feature engineering doesn't improved the overall prediction.**</span>

Even if we try to look for refurbished apartements or apartements with good heating we couldn't improve the prediction performance.

In [ ]:
# make a single binary variable to indicate if the apartment is refurbished/new
#df['refurbished'] = (df.condition == 'refurbished') | (df.condition == 'first_time_use') | \
#                    (df.condition == 'mint_condition') | (df.condition == 'fully_renovated') | \
#                    (df.condition == 'first_time_use_after_refurbishment')

#df = df.drop(['condition'], axis=1)
#df['refurbished']

In [ ]:
# make a binary variable to indicated if the rental property has good heating
#df['goodHeating'] = (df.heatingType == 'central_heating') | (df.heatingType == 'floor_heating') | \
#                    (df.heatingType == 'self_contained_central_heating')

#df = df.drop(['heatingType'], axis=1)
#df['goodHeating']

## Plots

In [ ]:
df.plot(subplots=True, layout=(3, 3), figsize=(12, 6), sharex=False, title="Feature Overview");

In [ ]:
df.plot(subplots=True, figsize=(10, 30));

Let's look at the correlation matrix again.

In [ ]:
# analyze feature importance
corr_matrix = df.corr()
corr_matrix['totalRent'].sort_values(ascending=False)

In [ ]:
report(df)

To save our clean data we export them as csv file.

In [ ]:
# export the cleaned data
df.to_csv("immo_data_clean.csv", encoding='utf-8', index=False)

# Training

Load the saved data.

In [ ]:
# load/read data
df = pd.read_csv("immo_data_clean.csv")
df_origin = df.copy()  # yes override this | from now on origin means cleaned data

# remaining features
features = list(df.columns)

Now let's split `totalRent` `y` from our dataset `X`.

In [ ]:
y = df['totalRent']
X = df.drop('totalRent', axis=1)
features.remove('totalRent')

Furthermore we will now split and scale the data.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

def scale(X, y, X_train, y_train, X_test):
    X_scaler, y_scaler = StandardScaler(), StandardScaler()

    X_scaled = X_scaler.fit_transform(X)
    y_scaled = y_scaler.fit_transform(
        y.values.reshape(-1, 1)
    ).flatten()

    X_train_scaled = pd.DataFrame(
        data=X_scaler.transform(X_train),
        columns=X.columns
    )
    y_train_scaled = y_scaler.transform(
        y_train.values.reshape(-1, 1)
    ).flatten()

    X_test_scaled = pd.DataFrame(
        data=X_scaler.transform(X_test),
        columns=X.columns
    )

    return [
        X_scaled, y_scaled,
        X_train_scaled, y_train_scaled,
        X_test_scaled,
        X_scaler, y_scaler
    ]

X_scaled, y_scaled, X_train_scaled, y_train_scaled, X_test_scaled, X_scaler, y_scaler = scale(X, y, X_train, y_train, X_test)

Let's define a little helper function to evaluate our models.

In [ ]:
from sklearn.metrics import mean_absolute_error
from pprint import pprint

def evaluate(model):
    def round_decimal(x):
        return f"{(int((x*100) + 0.5) / 100.0):0.2f}"

    predictions = model.predict(X_test_scaled)
    predictions = y_scaler.inverse_transform(predictions)

    errors = abs(predictions - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    mae = mean_absolute_error(y_test, predictions)  # np.mean(errors)
    min_error = np.min(errors)
    max_error = np.max(errors)

    print('Model Performance')
    print('=================')
    print(f"Mean Absolute Error : {round_decimal(mae):>10}€")
    print(f"Min. Error          : {round_decimal(min_error):>10}€")
    print(f"Max. Error          : {round_decimal(max_error):>10}€")
    print(f"Accuracy            : {round_decimal(accuracy):>10}%")

Now we will start to apply some regressors.

> `< 100k samples?`

In [ ]:
len(df)

Yes!

> Few features are important?

In [ ]:
len(df.columns)

Yes!

That means that we start with ElasticNet and Lasso.

## ElasticNet

In [ ]:
from sklearn.linear_model import ElasticNetCV

en_regr = ElasticNetCV(cv=5, random_state=42)
en_regr.fit(X_train_scaled, y_train_scaled)

evaluate(en_regr)

## Lasso

In [ ]:
from sklearn.linear_model import LassoCV

l_regr = LassoCV(cv=5, random_state=42)
l_regr.fit(X_train_scaled, y_train_scaled)

evaluate(l_regr)

Both models are working quite well. Before we start the hyperparameter tuning we want to try some additional models.

## Random Forest

Let's check if an ensemble Regressor could add some impremovents.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr_regr = RandomForestRegressor(random_state=42)
rfr_regr.fit(X_train_scaled, y_train_scaled)

evaluate(rfr_regr)

Despite the fewer features, Random Forest seems to reach the better performance.

Hence we will try to optimize this regressor (hyperparameter-tuning).

### Cross Validate

In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(
    rfr_regr,
    X_train_scaled,
    y_train_scaled,
    cv=5,
    scoring=('neg_mean_absolute_error'),
    return_train_score=True,
    return_estimator=True
)
#pprint(scores)

min_index = np.argwhere(np.abs(scores['train_score']) == np.min(np.abs(scores['train_score'])))
min_index = np.min(min_index)
rfr_regr = scores['estimator'][min_index]

evaluate(rfr_regr)

### RandomizedSearch

In [ ]:
n_estimators      = [ int(x) for x in np.linspace(start=200, stop=2000, num=10) ]
max_features      = [ 'auto', 'sqrt' ]
max_depth         = [ None ] + [ int(x) for x in np.linspace(10, 110, num=11) ]
min_samples_split = [ 2, 5, 10 ]
min_samples_leaf  = [ 1, 2, 4 ]
bootstrap         = [ True, False ]

random_grid = {
    'n_estimators'      : n_estimators,
    'max_features'      : max_features,
    'max_depth'         : max_depth,
    'min_samples_split' : min_samples_split,
    'min_samples_leaf'  : min_samples_leaf,
    'bootstrap'         : bootstrap
}

pprint(random_grid)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

rfr_regr = RandomForestRegressor()

rf_random = RandomizedSearchCV(
    estimator=rfr_regr,
    param_distributions=random_grid,
    n_iter=100,
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

rf_random.fit(X_train_scaled, y_train_scaled)

pprint(rf_random.best_params_)
evaluate(rf_random.best_estimator_)

### Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap'         : [ True ],
    'max_depth'         : [ 80, 90, 100, 110 ],
    'max_features'      : [ 2, 3 ],
    'min_samples_leaf'  : [ 3, 4, 5 ],
    'min_samples_split' : [ 8, 10, 12 ],
    'n_estimators'      : [ 100, 200, 300, 1000 ]
}

rfr_regr = RandomForestRegressor()

grid_search = GridSearchCV(
    estimator=rfr_regr,
    param_grid=param_grid,
    cv=3,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train_scaled, y_train_scaled)

pprint(grid_search.best_params_)
evaluate(grid_search.best_estimator_)

With over 87% we already do have a well trained model. But let's see if we can still find a better one.

## Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gb_regr = GradientBoostingRegressor(random_state=42)
gb_regr.fit(X_train_scaled, y_train_scaled)

evaluate(gb_regr)

### Randomized Search

In [ ]:
n_estimators      = [ int(x) for x in np.linspace(start=200, stop=2000, num=10) ]
max_features      = [ 'auto', 'sqrt' ]
max_depth         = [ None ] + [ int(x) for x in np.linspace(10, 110, num=11) ]
min_samples_split = [ 2, 5, 10 ]
min_samples_leaf  = [ 1, 2, 4 ]
learning_rate     = [ 0.25, 0.2, 0.15, 0.1, 0.05, 0.01 ]


random_grid = {
    'n_estimators'      : n_estimators,
    'max_features'      : max_features,
    'max_depth'         : max_depth,
    'min_samples_split' : min_samples_split,
    'min_samples_leaf'  : min_samples_leaf,
    'learning_rate'     : learning_rate
}

pprint(random_grid)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

gb_regr = GradientBoostingRegressor()

rf_random = RandomizedSearchCV(
    estimator=gb_regr,
    param_distributions=random_grid,
    n_iter=100,
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

rf_random.fit(X_train_scaled, y_train_scaled)

pprint(rf_random.best_params_)
evaluate(rf_random.best_estimator_)

##  Stochastic Gradient Descent (SGD)

In [ ]:
from sklearn.linear_model import SGDRegressor

sgd_regr = SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.1, random_state=42)
sgd_regr.fit(X_train_scaled, y_train_scaled)

evaluate(sgd_regr)

## Support Vector

In [ ]:
from sklearn.svm import SVR

sv_regr = SVR(kernel = 'rbf')
sv_regr.fit(X_train_scaled, y_train_scaled)

evaluate(sv_regr)

### Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'kernel' : [ 'linear', 'poly', 'rbf', 'sigmoid' ],
    'degree' : [ 2, 3, 4, 5 ],
    'C'      : [ 0.1, 0.5, 1.0, 2.0 ]
}

sv_regr = SVR()

grid_search = GridSearchCV(
    estimator=sv_regr,
    param_grid=param_grid,
    cv=3,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train_scaled, y_train_scaled)

pprint(grid_search.best_params_)
evaluate(grid_search.best_estimator_)

## AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

ada_regr = AdaBoostRegressor(random_state=42, n_estimators=100)
ada_regr.fit(X_train_scaled, y_train_scaled)

evaluate(ada_regr)

## Extreme Gradient Boosting

In [ ]:
from xgboost import XGBRegressor

model = XGBRegressor(objective='reg:squarederror')
model.fit(X_train_scaled, y_train_scaled)

evaluate(model)

### Grid Search

In [ ]:
parameters = {
    'nthread'          : [ 4 ],  # when use hyperthread, xgboost may become slower
    'objective'        : [ 'reg:squarederror' ],
    'learning_rate'    : [ 0.03, 0.05 ],  # so called `eta` value
    'max_depth'        : [ 5, 6 ],
    'min_child_weight' : [ 4 ],
    'subsample'        : [ 0.7 ],
    'colsample_bytree' : [ 0.7 ],
    'n_estimators'     : [ 500, 1000 ]
}

xgb1 = XGBRegressor()
xgb_grid = GridSearchCV(
    xgb1,
    parameters,
    cv=2,
    n_jobs=-1,
    verbose=True
)

xgb_grid.fit(X_train_scaled, y_train_scaled)

pprint(xgb_grid.best_params_)
evaluate(xgb_grid.best_estimator_)

## Light Gradient Boosting Machine

In [ ]:
from lightgbm import LGBMRegressor

model = LGBMRegressor(random_state=42)
model.fit(X_train_scaled, y_train_scaled)

evaluate(model)

In [ ]:
from lightgbm import LGBMRegressor

parameters = {
    'num_iter'      : [ 50, 100,  500, 1000 ],
    'learning_rate' : [ 0.25, 0.1, 0.05, 0.005 ],
    'n_estimators'  : [ 25, 100, 1000, 2000 ],
    'max_bin'       : [ 500, 1000 ],
}

model = LGBMRegressor(random_state=42)
lgbm_grid = GridSearchCV(
    model,
    parameters,
    cv=2,
    n_jobs=-1,
    verbose=True
)

lgbm_grid.fit(X_train_scaled, y_train_scaled)

pprint(lgbm_grid.best_params_)
evaluate(lgbm_grid.best_estimator_)

Light Gradient Boosting Machine not only work with an amazing speed but also has some nice build in features.

In [ ]:
from lightgbm import plot_importance
plot_importance(booster=lgbm_grid.best_estimator_)

# Result

| Model | MAE | Min | Max | Accuracy |
|--|--:|--:|--:|--:|
|ElasticNet|291.32€|0.30€|2066.67€|83.52%|
|Lasso|291.24€|0.58€|2067.44€|83.52%|
|Random Forest|236.47€|0.34€|1461.27€|86.80%|
|Random Forest (CV)|237.88€|0.36€|1323.56€|86.67%|
|Random Forest (RS)|232.06€|0.64€|1380.04€|87.05%|
|Random Forest (GS)|233.54€|0.49€|1400.27€|87.01%|
|Gradient Boosting|228.67€|0.31€|<span style="color: red">**1234.59€**</span>|87.43%|
|Gradient Boosting (RS)|229.08€|0.60€|1473.02€|87.30%|
|Stochastic Gradient Descent|301.45€|0.48€|1854.29€|82.54%|
|Support Vector|249.74€|0.06€|1759.94€|86.26%|
|Support Vector (GS)|249.74€|0.06€|1759.94€|86.26%|
|AdaBoost|329.29€|0.64€|1776.35€|79.99%|
|Extreme Gradient Boosting|235.19€|0.72€|1474.05€|86.97%|
|**Extreme Gradient Boosting (GS)**|<span style="color: red">**220.39€**</span>|0.62€|1353.08€|<span style="color: red">**87.82%**</span>|
|Light Gradient Boosting Machine|224.46€|<span style="color: red">**0.02€**</span>|1498.28€|87.69%|
|Light Gradient Boosting Machine (GS)|229.48€|0.06€|1401.13€|87.35%|

Legend:  
CV: Cross Validation  
RS: Randomized Search  
GS: Grid Search

There are a few well performing models. With a small lead Extreme Gradient Boosting Regressor is the winner.

In [ ]:
from xgboost import XGBRegressor

model = XGBRegressor(
    colsample_bytree=0.7,
    learning_rate=0.03,
    max_depth=5,
    min_child_weight=4,
    n_estimators=500,
    nthread=4,
    objective='reg:squarederror',
    subsample=0.7,
)
model.fit(X_train_scaled, y_train_scaled)

evaluate(model)

An interessing part could be to see the feature importance.

In [ ]:
predictors=list(X_train)
feat_imp = pd.Series(model.feature_importances_, predictors).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Importance of Features')
plt.xticks(rotation=45)
plt.ylabel('Feature Importance Score')

## Prediction Application

Now let's create a little prediction application for practical usage.

In [ ]:
# style
style = { 'description_width': '150px' }

# living space (sqm)
living_space = widgets.IntText(description='living space (sqm):', value=int(np.mean(df_origin['livingSpace'])), style=style)
living_space2 = widgets.IntSlider(min=np.min(df_origin['livingSpace']), max=np.max(df_origin['livingSpace']), readout=False)
widgets.jslink((living_space, 'value'), (living_space2, 'value'))
living_space_grid = TwoByTwoLayout(top_left=living_space, top_right=living_space2, width="650px")
display(living_space_grid)

# No. of rooms
no_rooms = widgets.IntText(description='no. of rooms:', value=int(np.mean(df_origin['noRooms'])), style=style)
no_rooms2 = widgets.IntSlider(min=np.min(df_origin['noRooms']), max=np.max(df_origin['noRooms']), readout=False)
widgets.jslink((no_rooms, 'value'), (no_rooms2, 'value'))
no_rooms_grid = TwoByTwoLayout(top_left=no_rooms, top_right=no_rooms2, width="650px")
display(no_rooms_grid)

# service charge
service_charge = widgets.FloatText(description='service charge:', value=int(np.mean(df_origin['serviceCharge'])*100)/100.0, style=style)
service_charge2 = widgets.FloatSlider(min=np.min(df_origin['serviceCharge']), max=np.max(df_origin['serviceCharge']), readout=False)
widgets.jslink((service_charge, 'value'), (service_charge2, 'value'))
service_charge_grid = TwoByTwoLayout(top_left=service_charge, top_right=service_charge2, width="650px")
display(service_charge_grid)

# picture count
picturecount = widgets.IntSlider(
    min=np.min(df_origin['picturecount']), max=np.max(df_origin['picturecount']),
    value=int(np.mean(df_origin['picturecount'])), description='picture count:', style=style
)

# No. of parking spaces
no_park_spaces = widgets.IntText(description='no. of parking spaces:', value=int(np.mean(df_origin['noParkSpaces'])), style=style)
no_park_spaces2 = widgets.IntSlider(min=np.min(df_origin['noParkSpaces']), max=np.max(df_origin['noParkSpaces']), readout=False)
widgets.jslink((no_park_spaces, 'value'), (no_park_spaces2, 'value'))
no_park_spaces_grid = TwoByTwoLayout(top_left=no_park_spaces, top_right=no_park_spaces2, width="650px")
display(no_park_spaces_grid)

# Post Code
geo_plz = widgets.Dropdown(
    options=sorted(set(df_origin['geo_plz'])),
    value=Counter(df['geo_plz']).most_common(1)[0][0],
    description='post code:',
    disabled=False,
    style=style,
)
display(geo_plz)

# Go button
def on_button_clicked(b):
    data = {
        'serviceCharge' : service_charge.value,
        'picturecount'  : picturecount.value,
        'noParkSpaces'  : no_park_spaces.value,
        'livingSpace'   : living_space.value,
        'geo_plz'       : geo_plz.value,
        'noRooms'       : no_rooms.value,
    }
    data_scaled = pd.DataFrame(
        data=X_scaler.transform([list(data.values())]),
        columns=list(data.keys())
    )
    predict = y_scaler.inverse_transform(model.predict(data_scaled))[0]
    prediction.value = f"Result: {(int((predict*100) + 0.5) / 100.0):0.2f}€"
btnGo = widgets.Button(
    description='Predict',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Start prediction!',
    icon='legal',
)
btnGo.on_click(on_button_clicked)
display(btnGo)
# prediction label
prediction = widgets.Label(value="Result: ")
display(prediction)